# Web Scraping All Book Titles From Wiki
##### for Dataset

#

___Importing all necessary libraries___

In [1]:
from bs4 import BeautifulSoup as bs
import requests as r
import sqlite3 as sq
import pandas as pd
import time

___Creating a function to store data in a db file.___

In [2]:
def db(list_category, book_title, author_name, date, book_link, author_link, link):
    conn = sq.connect('every_book_in_the_world.db')
    cor = conn.cursor()
    cor.executescript("""CREATE TABLE IF NOT EXISTS books(
        id INTEGER PRIMARY KEY NOT NULL,
        list_category TEXT,
        book_title TEXT unique,
        author_name TEXT,
        date TEXT,
        book_link TEXT,
        author_link TEXT,
        link TEXT);""")
    cor.execute("""INSERT OR IGNORE INTO books(list_category, book_title, author_name, date, book_link, author_link, link)
        VALUES(?, ?, ?, ?, ?, ?, ?)""", (list_category, book_title, author_name, date, book_link, author_link, link))
    conn.commit()
    print('Data committed')
    return None 

___Creating function to store links of categories of books___

In [3]:
def listdb(book_category, category_list_link):
    #import sqlite3 as sq
    #import time
    conn = sq.connect('list_of_books.db')
    cor = conn.cursor()
    cor.executescript("""CREATE TABLE IF NOT EXISTS list(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        book_category TEXT unique,
        category_list_link TEXT);""")
    cor.execute("""INSERT OR IGNORE INTO list(book_category, category_list_link)
                values(?, ?)""", (book_category, category_list_link))
    conn.commit()
    conn.close()
    return None
    

___Connection Function___

In [4]:
def loadsite(url):
    site = r.get(url)
    if site.status_code == 200:
        soup = bs(site.text, 'lxml')
        print(f'Status code: {site.status_code}')
        return soup

In [ ]:
soup = loadsite('https://en.wikipedia.org/wiki/Lists_of_books#Other_lists')
main_div = soup.findAll('div', class_='div-col')

In [ ]:
for dev in main_div:
    li = dev.findAll('li')
    for category_list_link in li:
        book_category = category_list_link.a.text
        category_list_link = 'https://en.wikipedia.org' + category_list_link.a['href']
        listdb(book_category, category_list_link)
        print(f'Title: {book_category} \n Link: {category_list_link}\n')

__Function To Find lists & Tables__

In [ ]:
def find_li(soup, div):
    count = 0
    divv = soup.find('div', class_=f'{div}')
    df = pd.DataFrame(columns=['list_category', 'book_title', 'author_name', 'date', 'book_link', 'author_link', 'link'])
    for li in divv.findAll('li'):
        try:
            Nan = ['Authors', 'Titles', 'Illustrators', 'Publishers', 'Category', 'Links']
            li = li.contents
            try:
                book_title = book_title.a.title.text.strip()
                
            ######
            try:
                book_link = 'https://en.wikipedia.org' + li[0].a['href']
            except:
                book_link = 'N/A'
            ######
            try:
                date = int(li[1].text.strip().split(' ')[0].split('(')[1].split(')')[0])
            except:
                date = 'N/A'
            ######
            try:
                author = li[2].text
                for i in author:
                    if i.isdigit():
                        break
                    if i in Nan:
                        break
            except:
                author = 'N/A'
            ######
            try:
                author_link = 'https://en.wikipedia.org' + li[2].a['href']
            except:
                author_link = 'N/A'
            ######
            count += 1
        except:
            continue
        df = df.append({'book_title': book_title, 'author_name': author, 'date': date, 'book_link': book_link, 'author_link': author_link}, ignore_index=True)
        #print(f'Book Title: {book_title} \n Book Link: {book_link} \n Date: {date} \n Author: {author} \n Author Link: {author_link} \n')
    return df

In [ ]:
#site ='https://en.wikipedia.org/wiki/List_of_18th-century_British_children%27s_literature_titles'
div = 'mw-parser-output'
conn = sq.connect('list_of_books.db')
corr = conn.cursor()
qdf = pd.read_sql_query("SELECT book_category, category_list_link FROM list", conn)
links = qdf['category_list_link']
count = 0


for site in links:
    time.sleep(1)
    soup = loadsite(site)
    var = find_li(soup, div)
    bc = qdf['book_category']
    bt = var['book_title']
    an = var['author_name']
    d = var['date']
    bl = var['book_link']
    al = var['author_link']
    for bc, bt, an, d, bl, al in zip(bc, bt, an, d, bl, al):
        count += 1
        db(bc, bt, an, d, bl, al, site)
print('Done')
conn.close()

In [7]:
conn = sq.connect('list_of_books.db')
corr = conn.cursor()
qdf = pd.read_sql_query("SELECT book_category, category_list_link FROM list", conn)
links = qdf['category_list_link']
links[0]

'https://en.wikipedia.org/wiki/List_of_18th-century_British_children%27s_literature_titles'

In [8]:
div = 'mw-parser-output'
soup = loadsite(links[0])

Status code: 200


In [17]:
count = 0
divv = soup.find('div', class_=f'{div}')
df = pd.DataFrame(columns=['list_category', 'book_title', 'author_name', 'date', 'book_link', 'author_link', 'link'])
for li in divv.findAll('li'):
    li = li.contents
    book_title = li[0].text.strip()    
    ######
    try:
        book_link = 'https://en.wikipedia.org' + li[0].a['href']
    except:
        book_link = 'N/A'
    ######
    try:
        date = int(li[1].text.strip().split(' ')[0].split('(')[1].split(')')[0])
    except:
        date = 'N/A'
    ######
    try:
        author = li[2].text
        for i in author:
            if i.isdigit():
                break
            if i in Nan:
                break
    except:
        author = 'N/A'
    ######
    try:
        author_link = 'https://en.wikipedia.org' + li[2].a['href']
    except:
        author_link = 'N/A'
    ######
    count += 1
    df = df.append({'book_title': book_title, 'author_name': author, 'date': date, 'book_link': book_link, 'author_link': author_link}, ignore_index=True)

In [19]:
df.to_csv('books.csv', index=False)